# Biophysics Lab - 2/15/2019

## Author: Stephanie M. Yan

Implement the Needleman-Wunsch alignment algorithm to align the following two sequences, using the HOXD70 scoring matrix and a gap penalty of 300.

In [1]:
seq1 = 'CATAAACCCTGGCGCGCTCGCGGCCCGGCACTCTTCTGGTCCCCACAGACTCAGAGAGAACCCACCATGGTGCTGTCTCCTGCCGACAAGACCAACGTCAAGGCCGCCTGGGGTAAGGTCGGCGCGCACGCTGGCGAGTATGGTGCGGAGGCCCTGGAGAGGATGTTCCTGTCCTTCCCCACCACCAAGACCTACTTCCCGCACTTCGACCTGAGCCACGGCTCTGCCCAGGTTAAGGGCCACGGCAAGAAGGTGGCCGACGCGCTGACCAACGCCGTGGCGCACGTGGACGACATGCCCAACGCGCTGTCCGCCCTGAGCGACCTGCACGCGCACAAGCTTCGGGTGGACCCGGTCAACTTCAAGCTCCTAAGCCACTGCCTGCTGGTGACCCTGGCCGCCCACCTCCCCGCCGAGTTCACCCCTGCGGTGCACGCCTCCCTGGACAAGTTCCTGGCTTCTGTGAGCACCGTGCTGACCTCCAAATACCGTTAAGCTGGAGCCTCGGTGGCCATGCTTCTTGCCCCTTGGGCCTCCCCCCAGCCCCTCCTCCCCTTCCTGCACCCGTACCCCCGTGGTCTTTGAATAAAGTCTGAGTGGGCGGCAAAAAAAAAAAAAAAAAAAAAA'
seq2 = 'GGGGCTGCCAACACAGAGGTGCAACCATGGTGCTGTCCGCTGCTGACAAGAACAACGTCAAGGGCATCTTCACCAAAATCGCCGGCCATGCTGAGGAGTATGGCGCCGAGACCCTGGAAAGGATGTTCACCACCTACCCCCCAACCAAGACCTACTTCCCCCACTTCGATCTGTCACACGGCTCCGCTCAGATCAAGGGGCACGGCAAGAAGGTAGTGGCTGCCTTGATCGAGGCTGCCAACCACATTGATGACATCGCCGGCACCCTCTCCAAGCTCAGCGACCTCCATGCCCACAAGCTCCGCGTGGACCCTGTCAACTTCAAACTCCTGGGCCAATGCTTCCTGGTGGTGGTGGCCATCCACCACCCTGCTGCCCTGACCCCGGAGGTCCATGCTTCCCTGGACAAGTTCTTGTGCGCCGTGGGCACTGTGCTGACCGCCAAGTACCGTTAAGACGGCACGGTGGCTAGAGCTGGGGCCAACCCATCGCCAGCCCTCCGACAGCGAGCAGCCAAATGAGATGAAATAAAATCTGTTGCATTTGTGCTCCAG'

## Initialize scoring matrix.

In [2]:
import numpy as np

rows = len(seq1) + 1
cols = len(seq2) + 1

F = np.zeros((rows,cols))
T = np.zeros((rows,cols))

# gap penalty
delt = 300

# initialize matriz
irows = 1
icols = 1

while icols < cols:
    F[0,icols] = F[0, icols-1] - delt
    icols += 1
while irows < rows:
    F[irows,0] = F[irows-1, 0] - delt
    irows += 1
    
print(F)

[[      0.    -300.    -600. ... -165600. -165900. -166200.]
 [   -300.       0.       0. ...       0.       0.       0.]
 [   -600.       0.       0. ...       0.       0.       0.]
 ...
 [-187500.       0.       0. ...       0.       0.       0.]
 [-187800.       0.       0. ...       0.       0.       0.]
 [-188100.       0.       0. ...       0.       0.       0.]]


## Fill in scoring matrix for the two sequences, using the given sigma matrix and gap penalty. Record optimal path in a traceback matrix.

In [3]:
icols = 1
irows = 1

# HoxD70 alignment scoring matrix (Chiaromonte, Yap, Miller 2002)
#                     A     C     G     T
sig = np.array ( [ [  91, -114,  -31, -123 ],
                  [ -114,  100, -125,  -31 ],
                  [  -31, -125,  100, -114 ],
                  [ -123,  -31, -114,   91 ] ] )

# dictionary for matching up nucleotide with row/column of scoring matrix
sigdict = { 'A' : 0,
            'C' : 1,
            'G' : 2,
            'T' : 3 }

# define function to calculate sigma score for an 'align' step
def calc_sig(seq1_nuc, seq2_nuc):
    rowid = sigdict[seq1_nuc]
    colid = sigdict[seq2_nuc]
    return(sig[rowid, colid])

# iterate through matrix, calculating the alignment score and tracing path at each step
while irows < rows:
    
    while icols < cols:
        vert = F[irows-1, icols] - delt # score for moving vertically
        hor = F[irows, icols-1] - delt # score for moving horizontally
        diag = F[irows-1, icols-1] + calc_sig(seq1[irows-1], seq2[icols-1]) # score for aligning
        F[irows,icols] = max(vert,hor,diag) # max score = direction of movement
                
        # record path in T matrix
        if F[irows,icols] == vert:
            T[irows,icols] = 1
        elif F[irows,icols] == hor:
            T[irows,icols] = 2
        elif F[irows,icols] ==diag:
            T[irows,icols] = 3
        
        icols += 1
        
    irows += 1
    icols = 1

print(F)
print(T)

[[ 0.00000e+00 -3.00000e+02 -6.00000e+02 ... -1.65600e+05 -1.65900e+05
  -1.66200e+05]
 [-3.00000e+02 -1.25000e+02 -4.25000e+02 ... -1.65200e+05 -1.65500e+05
  -1.65800e+05]
 [-6.00000e+02 -3.31000e+02 -1.56000e+02 ... -1.64809e+05 -1.65109e+05
  -1.65409e+05]
 ...
 [-1.87500e+05 -1.87100e+05 -1.86700e+05 ...  4.10900e+03  4.50000e+03
   4.76900e+03]
 [-1.87800e+05 -1.87400e+05 -1.87000e+05 ...  3.80900e+03  4.20000e+03
   4.46900e+03]
 [-1.88100e+05 -1.87700e+05 -1.87300e+05 ...  3.50900e+03  3.90000e+03
   4.16900e+03]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 3. 2. ... 2. 2. 2.]
 [0. 3. 3. ... 2. 2. 2.]
 ...
 [0. 1. 1. ... 1. 1. 1.]
 [0. 1. 1. ... 1. 1. 1.]
 [0. 1. 1. ... 1. 1. 1.]]


## Go through the traceback matrix to print out the aligned sequences.

In [5]:
irows = len(seq1)
icols = len(seq2)

# record nucleotide alignment for each sequence
align1 = ''
align2 = ''

# trace back through path matrix to align sequences
while irows > 0 or icols > 0:
    # if you go vertical
    if T[irows,icols] == 1:
        align1 = "-" + align1
        align2 = seq1[irows-1] + align2
        irows -= 1
        
    # if you go horizontal
    elif T[irows,icols] == 2:
        align1 = seq2[icols-1] + align1
        align2 = "-" + align2
        icols -= 1
    
    # if you go diagonal
    elif T[irows,icols] == 3:
        align1 = seq2[icols-1] + align1
        align2 = seq1[irows-1] + align2
        irows -= 1
        icols -=1
    
    # once you hit the initialized first row/column
    elif T[irows,icols] == 0:
        if irows == 0:
            while icols != 0:
                align1 = seq2[icols-1] + align1
                align2 = "-" + align2
                icols -= 1
        if icols == 0:
            while irows != 0:
                align1 = "-" + align1
                align2 = seq1[irows-1] + align2
                irows -= 1

print(align1)
print(align2)

print("Alignment score: " + str(F[len(seq1),len(seq2)]))

----------GG-G-GCT-GC--C--A--AC-------------ACAGA----G-GTG--CA-ACCATGGTGCTGTCCGCTGCTGACAAGAACAACGTCAAGGGCATCTTCACCAAAATCGCCGGCCATGCTGAGGAGTATGGCGCCGAGACCCTGGAAAGGATGTTCACCACCTACCCCCCAACCAAGACCTACTTCCCCCACTTCGATCTGTCACACGGCTCCGCTCAGATCAAGGGGCACGGCAAGAAGGTAGTGGCTGCCTTGATCGAGGCTGCCAACCACATTGATGACATCGCCGGCACCCTCTCCAAGCTCAGCGACCTCCATGCCCACAAGCTCCGCGTGGACCCTGTCAACTTCAAACTCCTGGGCCAATGCTTCCTGGTGGTGGTGGCCATCCACCACCCTGCTGCCCTGACCCCGGAGGTCCATGCTTCCCTGGACAAGTTCTTGTGCGCCGTGGGCACTGTGCTGACCGCCAAGTACCGTTAAGACGG--CA-CGGTGGCTA-GAG-CTGGGGCC--AA-CCCATCGCCAGCCC-TCCGACA-G-C--G-A---GCAGCCAAATGAGATG-AAATAAAATCTG--T---TG-CATTTGTGCTCCAG---------
CATAAACCCTGGCGCGCTCGCGGCCCGGCACTCTTCTGGTCCCCACAGACTCAGAGAGAACCCACCATGGTGCTGTCTCCTGCCGACAAGACCAACGTCAAGGCCGCCTGGGGTAAGGTCGGCGCGCACGCTGGCGAGTATGGTGCGGAGGCCCTGGAGAGGATGTTCCTGTCCTTCCCCACCACCAAGACCTACTTCCCGCACTTCGACCTGAGCCACGGCTCTGCCCAGGTTAAGGGCCACGGCAAGAAGGTGGCCGACGCGCTGACCAACGCCGTGGCGCACGTGGACGACATGCCCAACGCGCTGTCCGCCCTGAGCGACCTGCACGCGCACAAGCTTCGGGTGGACCCGGTCAACTTCAAGCTCCTA